In [1]:
import plotly.express as plt 
import pandas as pd
import os
import numpy as np

In [2]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))

In [3]:
utilization = pd.read_csv("../../build/Scheduler/utilizations.csv",sep=";")
throughputs = pd.read_csv("../../build/Scheduler/throughputs.csv", sep=";")
meanWaits = pd.read_csv("../../build/Scheduler/meanWaits.csv",sep=";")
meanClients = pd.read_csv("../../build/Scheduler/meanClients.csv", sep=";")
visits = {}
visits["CPU"] = 250
visits["SWAP_IN"] = 2.5
visits["IO1"] = 16.25
visits["IO2"] = 6.25 

In [4]:
def assign(dataframe):
    dataframe["CPU"] = dataframe["S2"]
    dataframe["SWAP_IN"] = dataframe["S1"]
    dataframe["IO1"] = dataframe["S3"]
    dataframe["IO2"] = dataframe["S4"]
    pass

assign(meanClients)
assign(meanWaits)
assign(throughputs)
assign(utilization)

responseTimes = {}
responseTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
responseTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
responseTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
responseTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]
responseTimes["N"] = meanWaits["N"]


In [5]:
def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    data = fig.to_image("png")
    f = open("./Images/{}.png".format(name),"+wb")
    f.write(data)
    f.close()
    fig.show()
pass

In [6]:
produce_graph_and_save(utilization,"Utilizations","B/T")

In [7]:
produce_graph_and_save(meanWaits,"meanWaits","ms",True)

In [8]:
produce_graph_and_save(meanClients,"meanClients","n")

In [9]:
produce_graph_and_save(throughputs,"throughputs","C/T")

In [10]:
produce_graph_and_save(responseTimes,"Response_Times","ms",True)

In [28]:
newmat = matrix.astype(np.double)
old = newmat.astype(np.double)
for i in range(50):
    old = newmat.astype(np.double)
    newmat = old.astype(np.double) @ old.astype(np.double)
    pass
newmat
visits = [x for x in newmat.diagonal()]
for i in range(1,len(visits),1):
    visits[i] = visits[i]*(1/visits[0])
    pass
visits

[0.003680870961733796,
 2.5,
 250.00000000000003,
 16.249999999999996,
 6.25000000000001]